In [ ]:
# !pip install wilds

In [1]:
from wilds import get_dataset
from wilds.common.data_loaders import get_train_loader
import torchvision.transforms as transforms

In [2]:
# Load the full dataset, and download it if necessary
dataset = get_dataset(dataset="globalwheat", download=True)

You can also download the dataset manually at https://wilds.stanford.edu/downloads.


10286178304Byte [12:02, 14238242.61Byte/s]                               


Extracting data/globalwheat_v1.1/archive.tar.gz to data/globalwheat_v1.1

It took 12.91 minutes to download and uncompress the dataset.



In [3]:
trans = transforms.Compose(
        [transforms.Resize((448, 448)), transforms.ToTensor()]
    )

train_data = dataset.get_subset(
    "train",
    transform=trans,
)

In [4]:
train_loader = get_train_loader("standard", train_data, batch_size=16)

In [8]:
for i, data in enumerate(train_loader):
    if i == 0:
        print(len(data))
        break

3
